In [1]:
import pandas as pd
import re
df_root = pd.read_csv('C:/Users/lui-m/Documents/GitHub/DS_Notebooks/ComunidadeDS/FTC_Python/datasets/train.csv')
df_root.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [2]:
# Fazendo cópia do dataframe lido
df = df_root.copy()

# Removendo espaço da string sem o loop for
df.loc[:, 'ID'] = df.loc[:, 'ID'].str.strip()
df.loc[:, 'Delivery_person_ID'] = df.loc[:, 'Delivery_person_ID'].str.strip()
df.loc[:, 'Road_traffic_density'] = df.loc[:, 'Road_traffic_density'].str.strip()
df.loc[:, 'Type_of_order'] = df.loc[:, 'Type_of_order'].str.strip()
df.loc[:, 'Type_of_vehicle'] = df.loc[:, 'Type_of_vehicle'].str.strip()

# Excluindo as linhas com a idade dos entregadores vazia
linhas_nao_vazias = df['Delivery_person_Age'] != 'NaN '
df = df.loc[linhas_nao_vazias, :]

# Conversao de texto/categoria/string para numeros inteiros
df['Delivery_person_Age'] = df['Delivery_person_Age'].astype(int)

# Conversao de texto/categoria/string para numeros decimais
df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].astype(float)

# Conversao de texto para data
df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%d-%m-%Y')

# Remove as linhas da coluna multiple_deliveries que tenham o conteúdo igual a 'NqN ' e fazendo o cast para inteiro
linhas_nao_vazias = df['multiple_deliveries'] != 'NaN '
df = df.loc[linhas_nao_vazias, :]
df['multiple_deliveries'] = df['multiple_deliveries'].astype(int)

# Removendo texto de números e resetando index
df = df.reset_index(drop=True)
for i in range(len(df)):
    df.loc[i, 'Time_taken(min)'] = re.findall(r'\d+', df.loc[i, 'Time_taken(min)'])

### 1. Qual o número total de entregas cadastradas na base de dados?

In [3]:
numero_entregas = df['ID'].count()
print(f'O número total de entregas cadastradas na base de dados é de: {numero_entregas}')

O número total de entregas cadastradas na base de dados é de: 42805


### 2. Qual o número total de entregadores únicos cadastrados na base de dados?

In [4]:
entregadores_unicos = df['Delivery_person_ID'].count()
print(f'O número total de entregadores cadastrados na base de dados é de: {entregadores_unicos}')

O número total de entregadores cadastrados na base de dados é de: 42805


### 3. Qual a média de idade dos entregadores por cidade?

In [5]:
cols = ['Delivery_person_Age', 'City']

group_age_city = df.loc[:, cols].groupby(['City']).mean()
group_age_city

,Delivery_person_Age
City,
Metropolitian,29.792763
NaN,28.810860
Semi-Urban,32.565789
Urban,28.908480


### 4. Qual a mediana de idade dos entregadores por cidade?

In [6]:
group_age_city = df.loc[:, cols].groupby(['City']).median()
group_age_city

,Delivery_person_Age
City,
Metropolitian,30.0
NaN,28.0
Semi-Urban,32.0
Urban,28.0


### 5. Qual o número total de entregas feitas por densidade de tráfego?

In [7]:
cols = ['ID', 'Road_traffic_density']

group_entregas_trafego = df.loc[:, cols].groupby('Road_traffic_density').count()
group_entregas_trafego

,ID
Road_traffic_density,
High,4212
Jam,13480
Low,14605
Medium,10417
NaN,91


### 6. Qual a média das avaliações das entregas feitas por densidade de tráfego?

In [8]:
cols = ['Delivery_person_Ratings', 'Road_traffic_density']
media_entregas_trafego = df.loc[:, cols].groupby('Road_traffic_density').mean()
media_entregas_trafego

,Delivery_person_Ratings
Road_traffic_density,
High,4.653492
Jam,4.595395
Low,4.646780
Medium,4.660627
NaN,3.912088


### 7. Qual a quantidade de entregas feitas por cidade?

In [9]:
cols = ['ID', 'City']

entregas_cidade = df.loc[:, cols].groupby(['City']).count()
entregas_cidade

,ID
City,
Metropolitian,32031
NaN,1105
Semi-Urban,152
Urban,9517


### 8. Qual a média, mediana e desvio padrão das entregas feitas por cidade e por tipo de veículo?

In [10]:
cols = ['ID', 'City', 'Type_of_vehicle']

entregas_numero = df.loc[:, cols].groupby(['City', 'Type_of_vehicle']).count()
entregas_numero

ID
City           Type_of_vehicle        
Metropolitian  bicycle              15
               electric_scooter   2590
               motorcycle        18989
               scooter           10437
NaN            electric_scooter     88
               motorcycle          581
               scooter             436
Semi-Urban     electric_scooter      3
               motorcycle          140
               scooter               9
Urban          electric_scooter    807
               motorcycle         5300
               scooter            3410

In [11]:
# Vamos mudar a pergunta pois esta questão foi anulada:
# Média, mediana e desvio padrão das avaliações das entregas feitas por cidade e por tipo de veículos
cols = ['Delivery_person_Ratings', 'City', 'Type_of_vehicle']
media_cidade = df.loc[:, cols].groupby(['City', 'Type_of_vehicle'])['Delivery_person_Ratings'].mean()
media_cidade

City            Type_of_vehicle 
Metropolitian   bicycle             3.333333
                electric_scooter    4.635408
                motorcycle          4.619287
                scooter             4.625782
NaN             electric_scooter    4.701136
                motorcycle          4.633966
                scooter             4.708028
Semi-Urban      electric_scooter    4.566667
                motorcycle          4.487143
                scooter             4.544444
Urban           electric_scooter    4.668983
                motorcycle          4.656463
                scooter             4.685886
Name: Delivery_person_Ratings, dtype: float64

In [12]:
mediana_cidade = df.loc[:, cols].groupby(['City', 'Type_of_vehicle'])['Delivery_person_Ratings'].median()
mediana_cidade

City            Type_of_vehicle 
Metropolitian   bicycle             1.0
                electric_scooter    4.7
                motorcycle          4.7
                scooter             4.7
NaN             electric_scooter    4.7
                motorcycle          4.7
                scooter             4.7
Semi-Urban      electric_scooter    4.7
                motorcycle          4.6
                scooter             4.6
Urban           electric_scooter    4.7
                motorcycle          4.7
                scooter             4.7
Name: Delivery_person_Ratings, dtype: float64

In [13]:
desvio_padrao_cidade = df.loc[:, cols].groupby(['City', 'Type_of_vehicle'])['Delivery_person_Ratings'].std()
desvio_padrao_cidade

City            Type_of_vehicle 
Metropolitian   bicycle             2.581989
                electric_scooter    0.385926
                motorcycle          0.317049
                scooter             0.365678
NaN             electric_scooter    0.287476
                motorcycle          0.289126
                scooter             0.255612
Semi-Urban      electric_scooter    0.513160
                motorcycle          0.365380
                scooter             0.360940
Urban           electric_scooter    0.444517
                motorcycle          0.283246
                scooter             0.298215
Name: Delivery_person_Ratings, dtype: float64

### 9. Qual a primeira data de entrega por cada cidade?

In [14]:
cols = ['Order_Date', 'City']

primeira_data = df.loc[:, cols].groupby(['City']).max()
primeira_data

,Order_Date
City,
Metropolitian,2022-04-06
NaN,2022-04-06
Semi-Urban,2022-04-06
Urban,2022-04-06


### 10. Qual a última data de entrega por cada cidade?

In [15]:
ultima_data = df.loc[:, cols].groupby(['City']).min()
ultima_data

,Order_Date
City,
Metropolitian,2022-02-11
NaN,2022-02-11
Semi-Urban,2022-02-11
Urban,2022-02-11


### 11. Qual a média de avaliações das entregas feitas por condições climáticas?

In [16]:
cols = ['Delivery_person_Ratings', 'Weatherconditions']

media_entregas_clima = df.loc[:, cols].groupby(['Weatherconditions']).mean()
media_entregas_clima

,Delivery_person_Ratings
Weatherconditions,
conditions Cloudy,4.652190
conditions Fog,4.653549
conditions NaN,3.912088
conditions Sandstorms,4.613000
conditions Stormy,4.613369
conditions Sunny,4.658030
conditions Windy,4.617804


### 12. Qual o valor da avaliação mais baixa feita por tipo de condição climática e por densidade de tráfego?

In [17]:
cols = ['Delivery_person_Ratings', 'Weatherconditions', 'Road_traffic_density']

menor_avaliacao = df.loc[:, cols].groupby(['Weatherconditions', 'Road_traffic_density'])['Delivery_person_Ratings'].min()
menor_avaliacao

Weatherconditions      Road_traffic_density
conditions Cloudy      High                    4.0
                       Jam                     4.0
                       Low                     3.5
                       Medium                  4.0
conditions Fog         High                    4.0
                       Jam                     4.0
                       Low                     3.5
                       Medium                  4.0
conditions NaN         NaN                     1.0
conditions Sandstorms  High                    4.0
                       Jam                     3.5
                       Low                     4.0
                       Medium                  3.5
conditions Stormy      High                    4.0
                       Jam                     3.5
                       Low                     4.0
                       Medium                  3.5
conditions Sunny       High                    3.5
                       Jam            

### 13. Qual a média, mediana e desvio padrão das avaliações por cada tipo de condições de veículos?

In [18]:
cols = ['Delivery_person_Ratings', 'Vehicle_condition']

media_condicao_veiculos = df.loc[:, cols].groupby(['Vehicle_condition']).mean()
mediana_condicao_veiculos = df.loc[:, cols].groupby(['Vehicle_condition']).median()
desvio_condicao_veiculos = df.loc[:, cols].groupby(['Vehicle_condition']).std()
print(media_condicao_veiculos)
print(mediana_condicao_veiculos)
print(desvio_condicao_veiculos)

                   Delivery_person_Ratings
Vehicle_condition                         
0                                 4.609126
1                                 4.648569
2                                 4.646178
3                                 3.912088
                   Delivery_person_Ratings
Vehicle_condition                         
0                                      4.7
1                                      4.7
2                                      4.7
3                                      6.0
                   Delivery_person_Ratings
Vehicle_condition                         
0                                 0.298199
1                                 0.320140
2                                 0.322822
3                                 2.479464


### 14. Quantas entregas foram feitas no Festival?

In [21]:
df_festival = df.loc[df['Festival'] == 'Yes ']
print(f'O número de entregas feitas no Festival foram de: {df_festival.shape[0]}')

O número de entregas feitas no Festival foram de: 848


### 15. Qual a avaliação média das entregas feitas no Festival?

In [23]:
cols = ['Delivery_person_Ratings', 'Festival']

media_festival = df_festival.loc[:, cols].groupby(['Festival']).mean()
media_festival

,Delivery_person_Ratings
Festival,
Yes,4.491509


### 16. Qual a menor avaliação feita em uma entrega no Festival e por cidade?

In [25]:
cols = ['Delivery_person_Ratings', 'Festival', 'City']

media_cidade_festival = df_festival.loc[:, cols].groupby(['Festival', 'City']).min()
media_cidade_festival

Delivery_person_Ratings
Festival City                                   
Yes      Metropolitian                       1.0
         NaN                                 4.0
         Semi-Urban                          3.5
         Urban                               2.5

### 17. Qual a maior avaliação feita por tipo de pedido?

In [30]:
cols = ['Delivery_person_Ratings', 'Type_of_order']

maior_avaliacao_pedido = df.loc[:, cols].groupby(['Type_of_order']).max()
maior_avaliacao_pedido

,Delivery_person_Ratings
Type_of_order,
Buffet,6.0
Drinks,6.0
Meal,6.0
Snack,6.0


### 18. Qual a idade do entregador mais velho da planilha de dados?

In [33]:
df['Delivery_person_Age'].max()

50

### 19. Qual a idade do entregador mais novo da planilha de dados?

In [34]:
df['Delivery_person_Age'].min()

15

### 20. Quantas entregas foram feitas em cada condição climática?

In [37]:
cols = ['ID', 'Weatherconditions']

entregas_condicoes_climaticas = df.loc[:, cols].groupby(['Weatherconditions']).count()

entrega